<a href="https://colab.research.google.com/github/darrenjhsu/birdid/blob/master/Bird_ID_make_img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
datapath = '/content/gdrive/My Drive/Bird_ID_project/nabirds'

In [0]:
!ls /content/gdrive/My\ Drive/Bird_ID_project/nabirds

In [0]:
if __name__ == '__main__':
    # Read files
    file_dir = '/content/gdrive/My Drive/Bird_ID_project/nabirds/'
    images = pd.read_csv(file_dir + 'images.txt', sep=" ", header=None, names = ["image_id","image_name"])
    print(images)
    #imagesFrame = pd.DataFrame([images], columns = )
    print("read image")
    train_test_split = pd.read_csv(file_dir + 'train_test_split.txt', sep=" ", header=None, names = ["image_id","is_training"])
    #trainTestSplitFrame = pd.DataFrame([train_test_split],columns = )
    print("read split")
    image_size = pd.read_csv(file_dir + 'sizes.txt', sep=" ", header=None, names = ["image_id","image_width","image_height"])
    #imageSizeFrame = pd.DataFrame([bounding_box],column )
    print("read image size")
    classes = pd.read_csv(file_dir + 'classes2.txt', sep=" ", header=None, names = ["class_id", "class_name"])
    #classesFrame = pd.DataFrame([classes],columns)
    print("read classes")
    image_class_label = pd.read_csv(file_dir + 'image_class_labels.txt', sep=" ", header=None, names = ["image_id","class_id"])
    #imageClassLabelFrame = pd.DataFrame([image_class_label],columns)
    print("read image class label")
    hierarchy = pd.read_csv(file_dir + 'hierarchy.txt', sep=" ", header=None, names = ["child_class_id","parent_class_id"])
    #hierarchyFrame = pd.DataFrame([hierarchy],columns)
    print("read hierarchy")    
    bounding_box = pd.read_csv(file_dir + 'bounding_boxes.txt', sep=" ", header=None, names = ["image_id","bb_x","bb_y","bb_width","bb_height"])
    #boundingBoxFrame = pd.DataFrame([bounding_box],column = ["image_id","bb_x","bb_y","bb_width","bb_height"])
    print("read bounding boxes")
    
    
    #print(classes)
    
    # Concatenate to a full frame including
    # ['image name','class id','x','y','x_end ( = x + x_dim)','y_end']
    # 
    
    img_bbox = pd.merge(images, bounding_box, on="image_id")
    img_class_bbox = pd.merge(img_bbox,image_class_label,on="image_id")
    full_frame = pd.merge(img_class_bbox,classes,on="class_id")
    
    print(full_frame)
    

In [0]:
if __name__ == '__main__':
    WBNH = full_frame[full_frame['class_name'] == 'White-breasted_Nuthatch']
    NCAM = full_frame[full_frame['class_name'] == 'Northern_Cardinal_(Adult_Male)']
    
    for index, row in WBNH.iterrows():
        if (index == 29669):
            print(row['image_name'])
            img = file_dir + 'images/' + row['image_name']
#            print(img)
            fig,ax = plt.subplots(1)
            image = plt.imread(img)
            plt.imshow(image)
            rect = patches.Rectangle((row['bb_x'],row['bb_y']),row['bb_width'],row['bb_height'],linewidth=1,edgecolor='r',facecolor='none')
            ax.add_patch(rect)
#            plt.imshow(img)
            plt.show()

In [0]:
def make_npy_2(df, bird1, bird2, y_path, X_path):
    print(y_path)
    df = df[(df['class_name']==bird1) | (df['class_name']==bird2)]
    #print(df)
    var = df.iloc[0]['class_id']
    df['class_id_2'] = df['class_id'].apply(lambda x: 1 if x == var else 0)
    #print(df)
    print('Bird class = 1: ', df.iloc[0]['image_name'])
    print('Bird class = 0: ', df.iloc[-1]['image_name'])
    target = df['class_id_2'].values
    np.save(y_path, target)
    print(target)
    zipper = zip(df.image_name, df.bb_y, df.bb_height, df.bb_x, df.bb_width)
    new_x = []
    for i, z in enumerate(zipper):
        print(datapath + '/images/' + z[0])
        image = cv2.imread(datapath + '/images/' + z[0])
        cropped = image[z[1]:(z[1]+z[2]), z[3]:(z[3]+z[4])]
        new_x.append(paint_to_square(cropped, z)/255.0)
        print(i)
        if (i < 10):
            print(image.shape)
            print(cropped.shape)
            cropped = paint_to_square(cropped, z)
            fig,ax = plt.subplots(1)
            #plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
            plt.imshow(cropped)
            plt.show()

    X = np.array(new_x)
    np.save(X_path, X)
    return target, X
    
    
def make_npy_many(df, bird_list, y_path, X_path, L_path):
    # y_path = ground truth label
    # X_path = data
    print(y_path)
#     df = df[(df['class_name']==bird1) | (df['class_name']==bird2)]
    df = df[df['class_name'].isin(bird_list)]
    df['class_id_2'] = pd.Series(np.zeros(len(df)), index=df.index)
#     print(df)
    var = df.class_id.unique()
    print(var)
    for i, z in enumerate(var):
        df.loc[df['class_id']==z,'class_id_2'] = i
#         print(df[df['class_id']==z])
#     print(df)
        print('Now processing bird class = {}: {} '.format(i,df.loc[df['class_id']==z,'class_name'].iloc[0]))
#     print('Bird class = 0: ', df.iloc[-1]['image_name'])
    target = df['class_id_2'].values.astype(int)
    np.save(y_path, target)
#     print(target)
    zipper = zip(df.image_name, df.bb_y, df.bb_height, df.bb_x, df.bb_width)
    new_x = []
    for i, z in enumerate(zipper):
        print(datapath + '/images/' + z[0])
        image = cv2.imread(datapath + '/images/' + z[0])
        cropped = image[z[1]:(z[1]+z[2]), z[3]:(z[3]+z[4])]
        new_x.append(paint_to_square(cropped, z)/255.0)
        print(i)
        if (i < 10):
            print(image.shape)
            print(cropped.shape)
            cropped = paint_to_square(cropped, z)
            fig,ax = plt.subplots(1)
            #plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
            plt.imshow(cropped)
            plt.show()

    X = np.array(new_x)
    np.save(X_path, X)
    
    np.save(L_path, bird_list)
    
    return target, X


def paint_to_square(img, z):
    desired_size = 224
    old_size = img.shape[:2] # old_size is in (height, width) format

    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    # new_size should be in (width, height) format

    im = cv2.resize(img, (new_size[1], new_size[0]))

    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
        value=color)
    new_im = cv2.cvtColor(new_im, cv2.COLOR_BGR2RGB)
    return new_im

In [0]:
y, X = make_npy_many(full_frame, ['Osprey','White-breasted_Nuthatch', 'Northern_Cardinal_(Adult_Male)'],datapath + '/data/y_3.npy', datapath + '/data/X_3.npy')
#make_npy(df, 'Black_Throated_Sparrow', 'Harris_Sparrow', )

In [0]:
full_frame.loc[full_frame['class_id']==814,'class_name'].iloc[0]

In [0]:
Bird_list=full_frame[full_frame['class_id'].isin((full_frame.class_id.value_counts()==120).index[:25].tolist())].class_name.unique()
print(Bird_list)
print(len(Bird_list))

In [0]:
y, X = make_npy_many(full_frame, Bird_list,datapath + '/data/y_30.npy', datapath + '/data/X_30.npy', datapath + '/data/L_30.npy')
#make_npy(df, 'Black_Throated_Sparrow', 'Harris_Sparrow', )

# 3/22 TODO: 
1.  Prepare the input file as square
1.  Run CNN transfer learning (with MobilV2?) with WBNH and NCAM
1.  Run fine tuning





In [0]:
!ls /content/gdrive/My\ Drive/Bird_ID_project/nabirds/data